## run this cell before starting spark cluster connection (corrects module error)

In [104]:
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [105]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [106]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)

### select the data

In [107]:
df = spark.read.json("/user/sprenner/training_all_april.json")

In [108]:
df.describe().toPandas()

summary  account           accountHash         clientState  \
0   count  3231296               3231296             3231296   
1    mean     None  -6.862163155897912E7                None   
2  stddev     None  1.2148734164282193E9                None   
3     min      aad           -2144719344  CannotAuthenticate   
4     max   zzheng            2143592175         SystemError   

        clientStateHash                                            dataset  \
0               3231296                                            3231296   
1  -9.350691361360884E8                                               None   
2  2.5054777314131996E8                                               None   
3           -1922357674                           BBS_M1000_nominal_100418   
4            1269532611  valid3.110401.PowhegPythia_P2012_ttbar_nonallh...   

             datasetHash datasetScope      datasetScopeHash eventType  \
0                3231296      3231296               3231296   3231296   
1  -4.5230852327388145E7         None  -4.507730727803677E8      None   
2   1.2453018016670852E9         None  1.0869769945929074E9      None   
3            -2147475923    cond09_mc           -2145215727  download   
4             2147475364       valid3            2131974379  download   

  eventTypeHash eventVersion      eventVersionHash  \
0       3231296      3231296               3231296   
1    5.937808E7         None  -1.963768093741567E9   
2           0.0         None   3.851356602973206E8   
3      59378080       1.13.2           -2044360925   
4      59378080          api            1261702107   

                           filename          filenameHash  \
0                           3231296               3231296   
1                              None   -1742584.3853425994   
2                              None  1.2397161559696255E9   
3  AOD.05371635._000001.pool.root.1           -2147481484   
4            wjets_v1.15_small.root            2147482617   

        filesize_double               hits                     hostname  \
0               3231296            3231296                      3231296   
1   3.482938720038774E8  5.085354916417438                         None   
2  1.0850033994592447E9  35.12657136873029                         None   
3                   5.0                  1  130-199-21-196.dhcp.bnl.gov   
4       3.0587790406E10               1348    xenia2.nevis.columbia.edu   

           hostnameHash protocol         protocolHash  \
0               3231296  3231296              3231296   
1    9.92930343842449E7     None  4.964437819779924E8   
2  1.3049200474702454E9     None   9.39691729069515E8   
3           -2142788649     davs          -1607963002   
4            2143857082      srm           2128034789   

                 remoteSite        remoteSiteHash      scope  \
0                   3231296               3231296    3231296   
1                      None  3.4601683712934375E7       None   
2                      None  1.2936656207502983E9       None   
3           AGLT2_CALIBDISK           -2135244854  cond09_mc   
4  ZA-WITS-CORE_SCRATCHDISK            2141740830     valid3   

              scopeHash                           traceId  \
0               3231296                           3231296   
1  -4.625174831601209E8                          Infinity   
2   1.084347690419062E9                               NaN   
3           -2145215727  00000a90a2c841548def8b19b974a7bf   
4            2131974379  fffffa77425f474b9fa9d590bdc6e23c   

            traceIdHash                    traceIp            traceIpHash  \
0               3231296                    3231296                3231296   
1    -620846.8997167081                       None  -2.1964318650645006E8   
2  1.2399847555363293E9                       None   1.2257798827174745E9   
3           -2147483615  2001:1458:201:e3::100:107            -2143850866   
4            2147482933        ::ffff:95.103.58.33             2146432649   

     tra

In [ ]:
df

### Separating the features from the target variable

In [109]:
assembler = VectorAssembler(
    inputCols=["traceTimeentryUnix", "transferStart", "accountHash","clientStateHash","datasetHash", "datasetScopeHash",
               "eventTypeHash", "eventVersionHash","filenameHash", "hostnameHash", "protocolHash", "remoteSiteHash", 
               "scopeHash", "traceIdHash", "traceIpHash", "uuidHash", "filesize_double"],
    outputCol="features")

#linearRegression will look for a column called features
output = assembler.transform(df)

In [110]:
#output = output.select("hits", "features")

### Split the data into train and test sets

In [111]:
train_data, test_data = output.randomSplit([.8,.2],seed=1234)

train_data.count()

2585235

In [112]:
test_data.count()

646061

### perform linear regression

In [113]:
from pyspark.ml.regression import LinearRegression

#finetuning: https://spark.apache.org/docs/2.1.0/ml-tuning.html
# https://spark.apache.org/docs/2.1.0/api/python/pyspark.ml.html#pyspark.ml.regression.LinearRegression

lr = LinearRegression(labelCol="hits", maxIter=10)

In [114]:
linearModel = lr.fit(train_data)

In [115]:
linearModel.numFeatures

17

In [116]:
linearModel.summary.residuals

DataFrame[residuals: double]

In [117]:
linearModel.coefficients

DenseVector([-0.0, -0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, 0.0, 0.0])

In [118]:
linearModel.intercept

510.7045315113562

In [119]:
linearModel.summary.rootMeanSquaredError

26.46709957154506

In [120]:
linearModel.summary.r2

0.4380642018698404

### Generate predictions

In [121]:
predicted = linearModel.transform(test_data)
predicted.limit(5).toPandas()

account  accountHash clientState  clientStateHash  \
0     aad   1640979169        DONE       -878527175   
1     aad   1640979169        DONE       -878527175   
2     aad   1640979169        DONE       -878527175   
3     aad   1640979169        DONE       -878527175   
4     aad   1640979169        DONE       -878527175   

                                             dataset  datasetHash  \
0  group.phys-higgs.data15_13TeV.AllYear.TOPQ1.gr...  -1148211875   
1  group.phys-higgs.data15_13TeV.AllYear.TOPQ1.gr...  -1148211875   
2  group.phys-higgs.data15_13TeV.AllYear.TOPQ1.gr...  -1148211875   
3  group.phys-higgs.data16_13TeV.AllYear.TOPQ1.gr...   1800172771   
4  group.phys-higgs.mc16_13TeV.345673.PoPy8_ttH12...   1335749446   

       datasetScope  datasetScopeHash eventType  eventTypeHash eventVersion  \
0  group.phys-higgs       -1366280164  download       59378080      1.14.11   
1  group.phys-higgs       -1366280164  download       59378080      1.14.11   
2  group.phys-higgs       -1366280164  download       59378080      1.14.11   
3  group.phys-higgs       -1366280164  download       59378080      1.14.11   
4  group.phys-higgs       -1366280164  download       59378080      1.14.11   

   eventVersionHash                                    filename  filenameHash  \
0       -2044360925  group.phys-higgs.13463555._000010.out.root    1387822711   
1       -2044360925  group.phys-higgs.13463555._000014.out.root     152179205   
2       -2044360925  group.phys-higgs.13463555._000022.out.root     314757982   
3       -2044360925  group.phys-higgs.13463569._000181.out.root    -159811778   
4       -2044360925  group.phys-higgs.13497303._000012.out.root    2139511111   

   filesize_double  hits           hostname  hostnameHash protocol  \
0     1.946970e+06     3  marquark.in2p3.fr   -2013835931     root   
1     1.058363e+06     3  marquark.in2p3.fr   -2013835931      srm   
2     2.501786e+06     3  marquark.in2p3.fr   -2013835931      srm   
3     3.017482e+07     3  marquark.in2p3.fr   -2013835931     root   
4     4.262071e+09    10  marquark.in2p3.fr   -2013835931      srm   

   protocolHash                           remoteSite  remoteSiteHash  \
0   -1364943180                CERN-PROD_SCRATCHDISK      1227399662   
1     863299160               DESY-ZN_LOCALGROUPDISK       631780862   
2     863299160  UKI-SCOTGRID-GLASGOW_LOCALGROUPDISK       878131778   
3   -1364943180                CERN-PROD_SCRATCHDISK      1227399662   
4     863299160                 BNL-OSG2_SCRATCHDISK      1969294760   

              scope   scopeHash                           traceId  \
0  group.phys-higgs -1366280164  5313dc2ebd984c82a212ed2aa58f61ea   
1  group.phys-higgs -1366280164  ab26827fab884f4f8f667f48bc8a6efe   
2  group.phys-higgs -1366280164  9845c81d786b4e369573c7e952ea5dc7   
3  group.phys-higgs -1366280164  cc3038580cb7400bbda4b42c0560955e   
4  group.phys-higgs -1366280164  46f5b02ed0904f9e8026bad162d8ccf3   

   traceIdHash                traceIp  traceIpHash  traceTimeentryUnix  \
0  -1176565801  ::ffff:134.158.17.108    312916504        1.523640e+09   
1  -1910620838  ::ffff:134.158.17.108    312916504        1.523640e+09   
2   -259104101  ::ffff:134.158.17.108    312916504        1.523640e+09   
3  -1550723742  ::ffff:134.158.17.108    312916504        1.523641e+09   
4  -1144319424  ::ffff:134.158.17.108    312916504        1.523811e+09   

   transferStart                              uuid    uuidHash  \
0   1.523640e+09  7c6c648fba054275bf9821932eecb902  -175166515   
1   1.523640e+09  7c6c648fba054275bf9821932eecb902  -175166515   
2   1.523640e+09  7c6c648fba054275bf9821932eecb902  -175166515   
3   1.523641e+09  57b5719577bd4da588ddfc7f05a0b7ac  1513267953   
4   1.523810e+09  494b94a404de4835898047b21538e695   445060265   

                                            features  prediction  
0  [1523640270.18, 1523640269.73, 1640979169.0, -...    2.839264  
1  [1523640281.32, 1523640278.86, 1640979169.0, -

In [122]:
# https://stackoverflow.com/questions/35882446/spark-convert-data-frame-column-to-vectors-dense
#df_pd = predicted.toPandas()

In [123]:
#df_pd = df_pd.sort_values('prediction')
#df_pd = df_pd.reset_index(drop=True)
#df_pd

In [124]:
#plt.rcParams['agg.path.chunksize'] = 10000

In [125]:
#df_pd[['hits','prediction']].plot(figsize=(16,10))

In [126]:
df_mai = spark.read.json("/user/sprenner/training_all_mai.json")

In [127]:
df_mai.describe().toPandas()

summary   account          accountHash     clientState  \
0   count   5316524              5316524         5316524   
1    mean      None  5.676034578304038E7            None   
2  stddev      None  1.078238184743408E9            None   
3     min  aalfonsi          -2144719344  AttributeError   
4     max    zzheng           2143592175     SystemError   

        clientStateHash                                            dataset  \
0               5316524                                            5316524   
1  -8.556539364422513E8                                               None   
2   3.393227998953656E8                                               None   
3           -1913538301                                    13917493.events   
4            1269532611  valid2.361023.Pythia8EvtGen_A14NNPDF23LO_jetje...   

            datasetHash datasetScope      datasetScopeHash eventType  \
0               5316524      5316524               5316524   5316524   
1   8.513416665722266E7         None  -2.489378879578484E8      None   
2  1.2643520814403243E9         None  1.1592276362619174E9      None   
3           -2147477854    cond09_mc           -2145215727  download   
4            2147445885       valid2            2144116684  download   

          eventTypeHash eventVersion       eventVersionHash  \
0               5316524      5316524                5316524   
1            5.937808E7         None  -2.0386347870917923E9   
2  7.106997961343818E-9         None    1.266751576337941E8   
3              59378080       1.13.2            -2044360925   
4              59378080       1.16.1             1352846300   

                                            filename          filenameHash  \
0                                            5316524               5316524   
1                                               None    -576003.5649204254   
2                                               None  1.2387416216837683E9   
3                                              0.dat           -2147483353   
4  user.zzheng.410389.MadGraphPythia8EvtGen.DAOD_...            2147479805   

        filesize_double                hits                  hostname  \
0               5316524             5316524                   5316524   
1  2.8167336660135007E8   6.517046664324284                      None   
2  1.0094883391407571E9  13.920048576048561                      None   
3                   2.0                   1    ReCaS-ui-01.cs.infn.it   
4       4.7520668166E10                 249  xenia.nevis.columbia.edu   

           hostnameHash protocol         protocolHash  \
0               5316524  5316524              5316524   
1  1.8503396854168174E8     None  5.671483465013922E8   
2  1.1133988987325292E9     None  9.234807216961867E8   
3           -2140594599     davs          -1607963002   
4            2130161317      srm           2128034789   

                 remoteSite        remoteSiteHash      scope  \
0                   5316524               5316524    5316524   
1                      None  2.0771631249619377E8       None   
2                      None  1.2437185785805871E9       None   
3           AGLT2_CALIBDISK           -2113133296  cond09_mc   
4  ZA-WITS-CORE_SCRATCHDISK            2141740830     valid2   

               scopeHash                           traceId  \
0                5316524                           5316524   
1  -2.4953820189199993E8                          Infinity   
2   1.1593529202301002E9                               NaN   
3            -2145215727  00000696ec754c998f70d84d737dfd18   
4             2144116684  fffffe65b9c549e5ae746c15ec98d5bb   

            traceIdHash                    traceIp            traceIpHash  \
0               5316524                    5316524                5316524   
1   -254286.60280984343                       None  -4.1963757976758495E7   
2  1.2400714422504263E9                       None   1.2010453269587846E9   
3           -2147483583  2001:1458:201:e4::

In [128]:
assembler_mai = VectorAssembler(
    inputCols=["traceTimeentryUnix", "transferStart", "accountHash","clientStateHash","datasetHash", "datasetScopeHash",
               "eventTypeHash", "eventVersionHash","filenameHash", "hostnameHash", "protocolHash", "remoteSiteHash", 
               "scopeHash", "traceIdHash", "traceIpHash", "uuidHash", "filesize_double"],
    outputCol="features")

#linearRegression will look for a column called features
output_mai = assembler_mai.transform(df_mai)

In [129]:
predicted_mai = linearModel.transform(output_mai)

In [130]:
predicted_mai.describe().toPandas()

summary   account          accountHash     clientState  \
0   count   5316524              5316524         5316524   
1    mean      None  5.676034578304038E7            None   
2  stddev      None  1.078238184743408E9            None   
3     min  aalfonsi          -2144719344  AttributeError   
4     max    zzheng           2143592175     SystemError   

        clientStateHash                                            dataset  \
0               5316524                                            5316524   
1  -8.556539364422513E8                                               None   
2   3.393227998953657E8                                               None   
3           -1913538301                                    13917493.events   
4            1269532611  valid2.361023.Pythia8EvtGen_A14NNPDF23LO_jetje...   

            datasetHash datasetScope      datasetScopeHash eventType  \
0               5316524      5316524               5316524   5316524   
1   8.513416665722266E7         None  -2.489378879578484E8      None   
2  1.2643520814403243E9         None   1.159227636261917E9      None   
3           -2147477854    cond09_mc           -2145215727  download   
4            2147445885       valid2            2144116684  download   

          eventTypeHash eventVersion       eventVersionHash  \
0               5316524      5316524                5316524   
1            5.937808E7         None  -2.0386347870917916E9   
2  7.106731401323233E-9         None   1.2667515763379405E8   
3              59378080       1.13.2            -2044360925   
4              59378080       1.16.1             1352846300   

                                            filename         filenameHash  \
0                                            5316524              5316524   
1                                               None   -576003.5649204254   
2                                               None  1.238741621683768E9   
3                                              0.dat          -2147483353   
4  user.zzheng.410389.MadGraphPythia8EvtGen.DAOD_...           2147479805   

        filesize_double               hits                  hostname  \
0               5316524            5316524                   5316524   
1  2.8167336660135007E8  6.517046664324284                      None   
2  1.0094883391407571E9  13.92004857604856                      None   
3                   2.0                  1    ReCaS-ui-01.cs.infn.it   
4       4.7520668166E10                249  xenia.nevis.columbia.edu   

           hostnameHash protocol         protocolHash  \
0               5316524  5316524              5316524   
1  1.8503396854168174E8     None  5.671483465013922E8   
2  1.1133988987325294E9     None  9.234807216961868E8   
3           -2140594599     davs          -1607963002   
4            2130161317      srm           2128034789   

                 remoteSite        remoteSiteHash      scope  \
0                   5316524               5316524    5316524   
1                      None  2.0771631249619377E8       None   
2                      None   1.243718578580587E9       None   
3           AGLT2_CALIBDISK           -2113133296  cond09_mc   
4  ZA-WITS-CORE_SCRATCHDISK            2141740830     valid2   

               scopeHash                           traceId  \
0                5316524                           5316524   
1  -2.4953820189199993E8                          Infinity   
2   1.1593529202301002E9                               NaN   
3            -2145215727  00000696ec754c998f70d84d737dfd18   
4             2144116684  fffffe65b9c549e5ae746c15ec98d5bb   

           traceIdHash                    traceIp            traceIpHash  \
0              5316524                    5316524                5316524   
1  -254286.60280984343                       None  -4.1963757976758495E7   
2  1.240071442250426E9                       None   1.2010453269587848E9   
3          -2147483583  2001:1458:201:e4::100:28c          

In [131]:
#df_pd_mai = predicted_mai[['hits','prediction']].toPandas()

In [132]:
#df_pd_mai = df_pd_mai.sort_values('prediction')
#df_pd_mai = df_pd_mai.reset_index(drop=True)
#df_pd_mai.plot(figsize=(16,10))

In [133]:
predicted_mai=predicted_mai.withColumn("filesize", predicted_mai["filesize_double"].cast(IntegerType()))

In [134]:
predicted_mai = predicted_mai.drop('filesize_double')

## Save separately for plotting

In [135]:
df_mai_plot = predicted_mai.select('hits', 'prediction')
df_mai_plot.write.csv('/user/sprenner/linear_regression_plot_mai_all_predictions_maxIter10.csv', sep='\t')

In [103]:
df_mai_sim = predicted_mai.select('uuid', 'account', 'dataset', 'filename', 'eventType', 'clientState', 
                                  from_unixtime(predicted_mai.traceTimeentryUnix, format='yyyy-MM-dd').alias('day'), 'traceTimeentryUnix', 
                                  'filesize', 'prediction').sort(col("day").asc())

In [69]:
df_mai_sim.limit(5).toPandas()

uuid account  \
0  a71a723e9f9341c5a18bbdc09b4ee87e   cylin   
1  d792a34ec85c4311b4246e91e4c4a830   cylin   
2  a71a723e9f9341c5a18bbdc09b4ee87e   cylin   
3  a71a723e9f9341c5a18bbdc09b4ee87e   cylin   
4  a71a723e9f9341c5a18bbdc09b4ee87e   cylin   

                                             dataset  \
0  mc16_13TeV.361021.Pythia8EvtGen_A14NNPDF23LO_j...   
1  mc16_13TeV.361021.Pythia8EvtGen_A14NNPDF23LO_j...   
2  mc16_13TeV.361021.Pythia8EvtGen_A14NNPDF23LO_j...   
3  mc16_13TeV.361021.Pythia8EvtGen_A14NNPDF23LO_j...   
4  mc16_13TeV.361021.Pythia8EvtGen_A14NNPDF23LO_j...   

                           filename eventType         clientState         day  \
0  AOD.12911492._000076.pool.root.1  download  ServiceUnavailable  2018-05-01   
1  AOD.12911492._000076.pool.root.1  download                DONE  2018-05-01   
2  AOD.12911492._000076.pool.root.1  download  ServiceUnavailable  2018-05-01   
3  AOD.12911492._000076.pool.root.1  download  ServiceUnavailable  2018-05-01   
4  AOD.12911492._000076.pool.root.1  download  ServiceUnavailable  2018-05-01   

   traceTimeentryUnix    filesize  prediction  
0        1.525170e+09  2147483647    5.654871  
1        1.525201e+09  2147483647    4.188057  
2        1.525170e+09  2147483647    5.654869  
3        1.525170e+09  2147483647    5.654869  
4        1.525170e+09  2147483647    5.654864

In [70]:
df_mai_sim.write.csv('/user/sprenner/linear_regression_mai_all_predictions_maxIter10.csv', sep='\t')